In [11]:
# The below libaries are specific to the skeltonize processing function and the fil_finder library requires you use pip
import astropy.units
from fil_finder import FilFinder2D
import imageio
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import os
import pandas as pd
import pygmt
import shutil
from skimage.morphology import skeletonize, flood_fill
import sys
import xarray as xr

In [12]:
import skeletonFunctions

import importlib
importlib.reload(skeletonFunctions)


<module 'skeletonFunctions' from '/Users/jono/Library/CloudStorage/OneDrive-TheUniversityofSydney(Staff)/makeSkeltons/skel-code/skeletonFunctions.py'>

In [13]:
elevationsPath = '../data/paleoElevations'

In [6]:
outdir = 'skeletons'
if os.path.isdir(f'{outdir}'):
	shutil.rmtree(f'{outdir}')
os.makedirs(outdir)

for f in os.listdir(elevationsPath):
    if f.endswith('.nc'):
        grid = os.path.join(elevationsPath,f)
    else:
        continue

    if os.path.exists(f'{os.getcwd()}/{f}'):
        os.remove(f'{os.getcwd()}/{f}')

    shutil.copy(f'{grid}', os.path.join(os.getcwd(), f'{f}'))
    grid=f
    
    smoothMask = skeletonFunctions.makeSmoothMask(grid)
    landSkel = skeletonFunctions.generate_gateway(smoothMask, ocean_or_land='land')
    oceanSkel = skeletonFunctions.generate_gateway(smoothMask, ocean_or_land='ocean')

    shutil.move(landSkel,outdir)
    shutil.move(oceanSkel,outdir)
    os.remove(grid)

Identifying unique bodies of water
Floodfill of water bodies done
Now checking connectivity of water across grid edges
uh oh, there is connected water across the grid edges
fixing..
Detected 77 unique bodies of water
Looking at water body #1
This water body takes up 32738 cells
This water body is big enough to stay
Looking at water body #2
This water body takes up 35 cells
Min sea size is 500, so turning this to land
Looking at water body #3
This water body takes up 51 cells
Min sea size is 500, so turning this to land
Looking at water body #4
This water body takes up 17 cells
Min sea size is 500, so turning this to land
Looking at water body #5
This water body takes up 17 cells
Min sea size is 500, so turning this to land
Looking at water body #6
This water body takes up 5665 cells
This water body is big enough to stay
Looking at water body #7
This water body takes up 14 cells
Min sea size is 500, so turning this to land
Looking at water body #8
This water body takes up 15 cells
Min s

/Users/jono/opt/anaconda3/envs/skeletons/lib/python3.10/site-packages/fil_finder/filfinder2D.py:142: UserWarning: No beam width given. Using 0 pixels.
  warnings.warn("No beam width given. Using 0 pixels.")
/Users/jono/opt/anaconda3/envs/skeletons/lib/python3.10/site-packages/fil_finder/filfinder2D.py:296: UserWarning: Using inputted mask. Skipping creation of anew mask.
  warnings.warn("Using inputted mask. Skipping creation of a"
/Users/jono/opt/anaconda3/envs/skeletons/lib/python3.10/site-packages/fil_finder/filament.py:326: UserWarning: Graph pruning reached max iterations.
  warnings.warn("Graph pruning reached max iterations.")


Identifying unique bodies of water
Floodfill of water bodies done
Now checking connectivity of water across grid edges
uh oh, there is connected water across the grid edges
fixing..
Detected 69 unique bodies of water
Looking at water body #1
This water body takes up 27310 cells
This water body is big enough to stay
Looking at water body #2
This water body takes up 5463 cells
This water body is big enough to stay
Looking at water body #3
This water body takes up 6 cells
Min sea size is 500, so turning this to land
Looking at water body #4
This water body takes up 5220 cells
This water body is big enough to stay
Looking at water body #5
This water body takes up 39 cells
Min sea size is 500, so turning this to land
Looking at water body #6
This water body takes up 9353 cells
This water body is big enough to stay
Looking at water body #7
This water body takes up 29 cells
Min sea size is 500, so turning this to land
Looking at water body #8
This water body takes up 208 cells
Min sea size is

/Users/jono/opt/anaconda3/envs/skeletons/lib/python3.10/site-packages/fil_finder/length.py:282: RuntimeWarning: invalid value encountered in scalar divide
  w * (intensity[idx] / np.sum(intensity))


Identifying unique bodies of water
Floodfill of water bodies done
Now checking connectivity of water across grid edges
uh oh, there is connected water across the grid edges
fixing..
uh oh, there is connected water across the grid edges
fixing..
uh oh, there is connected water across the grid edges
fixing..
Detected 152 unique bodies of water
Looking at water body #1
This water body takes up 26553 cells
This water body is big enough to stay
Looking at water body #2
This water body takes up 1 cells
Min sea size is 500, so turning this to land
Looking at water body #3
This water body takes up 13 cells
Min sea size is 500, so turning this to land
Looking at water body #4
This water body takes up 2 cells
Min sea size is 500, so turning this to land
Looking at water body #5
This water body takes up 6 cells
Min sea size is 500, so turning this to land
Looking at water body #6
This water body takes up 23506 cells
This water body is big enough to stay
Looking at water body #7
This water body ta

In [15]:
# Hopefully all the gateways are sitting in a folder
# For each age there should be a gateway for land, and one for water
# Let's plot these to check they look ok

if os.path.exists('skeletonFigs'):
    shutil.rmtree('skeletonFigs')
os.makedirs('skeletonFigs')

proj="Q15c"
# proj="W0/15c"
region='d' # d = Global region
anot_spacing="a30f15g30"
w="1.5p,black" # default pen settings for drawing plate boundaries etc
bar="JBC+o0/1.2"

images = []
for f in os.listdir(elevationsPath):
    age = f.split('_')[-2]
    ocean_skel = f'skeletons/skeleton_ocean_topo_{age}_filled.nc'
    land_skel = f'skeletons/skeleton_land_topo_{age}_filled.nc'
    originalElevs = f'{elevationsPath}/{f}'

    fig = pygmt.Figure()

    etopo_cpt = pygmt.makecpt(cmap='etopo1', series=[-10000,10000,500])
    fig.grdimage(grid=f'{originalElevs}?z', cmap=etopo_cpt, projection=proj, region=region, frame=[f'+t"{age}Ma Gateways"', anot_spacing])
    fig.grdcontour(grid=f'{ocean_skel}?pruned_trimmed_skel_noDiagonal', interval='+0.5', pen='1.2p,black', region=region, projection=proj)
    fig.grdcontour(grid=f'{land_skel}?pruned_trimmed_skel_noDiagonal', interval='+0.5', pen='1.2p,red', region=region, projection=proj)
    fig.colorbar(position=bar, cmap=etopo_cpt, frame=["a4000f2000", "x+lElevation", "y+lm"])

    fig.savefig(f'skeletonFigs/{age.zfill(4)}_DEM.png')
    images.append(f'skeletonFigs/{age.zfill(4)}_DEM.png')

readims = []
for im in sorted(images):
    readims.append(imageio.v2.imread(im))
imageio.v2.mimwrite(f'skeletonFigs/animation.gif',readims,duration=2000)
